In [1]:
twitterRDD = sc.textFile("C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/data_big_text.csv").cache()

In [2]:
# Function to clean words
def normalizeWords(word):
    new_word = word.lower().replace("u'",'').replace('[', '').replace(']', '').replace(',', '').replace("'", '')\
        .replace('"','').replace("(",'').replace(")",'').replace("!",'').replace("?",'').replace(":",'')   
    if new_word != '':
        return new_word
    else:
        return 'NA'

In [3]:
twitterRDD_word = twitterRDD.flatMap(lambda line: line.split()).map(lambda word: normalizeWords(word)).map(lambda word: (word,1))

In [4]:
# Build stop words and marks
import re
stopWords = open('C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/stop-words.txt', 'r')
sWords = re.split(r'\n', stopWords.read())
marks = [',','-','&amp;','??','+','|','...',':','!','"']

In [5]:
# Filter words
twitterRDD_word_filtered = twitterRDD_word.filter(lambda (k,v) : k not in sWords).filter(lambda (k,v) : k not in marks)
twitterRDD_flat = twitterRDD_word_filtered.filter(lambda (k,v): k != 'NA').map(lambda (k,v): k)
ttl_word = twitterRDD_flat.count()

In [6]:
text = str(twitterRDD_flat.take(ttl_word)).split(',')
text = str(text).replace("u'",'').replace('[', '').replace(']', '').replace(',', '').replace("'", '')\
        .replace('"','').replace("(",'').replace(")",'').replace("!",'').replace("?",'').replace(":",'')

In [8]:
# Calculate bigrams
from nltk import ngrams
import pandas as pd
bigrams_all = ngrams(text.split('  '), 2)
bigramsDF = pd.DataFrame(bigrams_all)

In [11]:
bigramsDF['count'] = 1
bigramsDF.columns = ['w1', 'w2', 'count']
bigrams_count = bigramsDF.groupby(['w1', 'w2'])['count'].sum()
bigrams_countDF = pd.DataFrame(bigrams_count)
bigrams_countDF.sort(['count'], ascending=False)

C:\Users\byron\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,,count
w1,w2,
apple,music,13391
bid,now,9010
apple,iphone,8107
eli,apple,6497
@drake,#views,5020
available,itunes,4580
now,available,4448
apple,ipad,3931
#views,now,3829


In [12]:
# Save file
bigrams_countDF.to_csv("C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/bigrams_new.csv", sep=',') 